### 04. Model

This notebook creates the CNN + RNN model and saves two files:

1. `model.json`
2. `model.h5` (weights)

##### Requires
1. Variables from 3rd notebook
2. Embedding Matrix (`embedding_matrix.pkl`) from 3rd notebook

In [1]:
import pickle

from keras.models import Model,Input
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.layers import Embedding,Dense,BatchNormalization,Dropout,LSTM,add
from keras.utils import plot_model
from keras.models import model_from_json
from keras.models import load_model

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


Using TensorFlow backend.
/mnt/c/Users/Kanishk/Documents/Projects/sem6/wsl/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/c/Users/Kanishk/Documents/Projects/sem6/wsl/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/c/Users/Kanishk/Documents/Projects/sem6/wsl/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qi

In [2]:
def load_file(file_path):
    with open(file_path, "rb") as f:
        return pickle.load(f)

In [4]:
# standard variables
MAX_LENGTH = 47 # 52 for actual dataset
VOCAB_SIZE = 1539 # 6321 for actual dataset. Using unique words as vocab here.
NPIX = 299
TARGET_SIZE = (NPIX,NPIX,3)
EMBEDDING_SIZE = 300


In [5]:
# partial caption sequence model

inputs2 = Input(shape=(MAX_LENGTH,))
se1 = Embedding(VOCAB_SIZE, EMBEDDING_SIZE, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(300)(se2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
# image feature extractor model
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(300, activation='relu')(fe1)

In [7]:
decoder1 = add([fe2, se3])
decoder2 = Dense(300, activation='relu')(decoder1)
outputs = Dense(VOCAB_SIZE, activation='softmax')(decoder2)

In [8]:
# merge the two input models
# image_feature + partial caption ===> output
model = Model(inputs=[inputs1, inputs2], outputs=outputs)

In [9]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 52)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 52, 300)      1896300     input_1[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 2048)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [10]:
embedding_matrix = load_file('embedding_matrix.pkl')

In [11]:
print(model.layers[2])

In [12]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

In [13]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [14]:
# keras library import  for Saving and loading model and weights


model_json = model.to_json()

with open("model5000.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model5000.h5")